In [1]:
import numpy as np
import pandas as pd

import glam

%load_ext autoreload
%autoreload 2

/Users/armin/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# hierarchical models
## one factor
### one factor between subject

In [2]:
np.random.seed(1)

m1 = glam.GLAM()

groups = ['high', 'low']

N = dict(high=5, low=3)

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))

for group in groups:
    
    parameters = dict(v=np.clip(np.random.normal(loc=0.00007, scale=0.00001, size=N[group]), 0.00003, 0.00015),
                      s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N[group]), 0.005, 0.011),
                      gamma=np.clip(np.random.normal(loc=gamma[group]['mu'], scale=gamma[group]['sd'], size=N[group]), -1, 1),
                      tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N[group]), 0.2, 2),
                      t0=np.zeros(N[group]))
    
    m1.simulate_group(kind='individual',
                      parameters=parameters,
                      n_individuals=N[group],
                      n_trials=n_trials,
                      n_items=n_items,
                      label=group)
m1.data.rename(columns=dict(condition='group'), inplace=True)

m1.make_model(kind='hierarchical', 
              depends_on=dict(gamma='group'),
              t0_val=0)
m1.fit(method='ADVI', n_advi=10000)

Generating hierarchical model for 8 subjects...
Fitting 1 model(s) using ADVI...
  Fitting model 1 of 1...


Average Loss = 3,783.4: 100%|██████████| 10000/10000 [00:35<00:00, 285.29it/s]


/!\ Automatically setting parameter precision...


In [3]:
estimates = glam.utils.get_estimates(m1)
estimates

/!\ Automatically setting parameter precision...


,gamma,gamma_hpd_2.5,gamma_hpd_97.5,gamma_mu,gamma_mu_hpd_2.5,gamma_mu_hpd_97.5,group,s,s_hpd_2.5,s_hpd_97.5,...,tau_hpd_97.5,tau_mu,tau_mu_hpd_2.5,tau_mu_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5,v_mu,v_mu_hpd_2.5,v_mu_hpd_97.5
0,-0.31,-0.554735,0.164557,-0.234023,-0.718341,0.294359,high,0.009152,0.004533,0.013447,...,2.118060,1.290151,0.585978,2.060308,0.000067,0.000029,0.000128,0.000095,0.000039,0.000164
1,-0.63,-0.830529,-0.122980,-0.234023,-0.718341,0.294359,high,0.011250,0.006635,0.015298,...,2.000246,1.290151,0.585978,2.060308,0.000050,0.000029,0.000117,0.000095,0.000039,0.000164
2,-0.22,-0.759531,0.301233,-0.234023,-0.718341,0.294359,high,0.008546,0.006658,0.013828,...,2.230788,1.290151,0.585978,2.060308,0.000047,0.000016,0.000132,0.000095,0.000039,0.000164
3,-0.41,-0.802804,0.007287,-0.234023,-0.718341,0.294359,high,0.010645,0.007554,0.014476,...,2.202573,1.290151,0.585978,2.060308,0.000059,0.000023,0.000119,0.000095,0.000039,0.000164
4,-0.36,-0.635984,0.061544,-0.234023,-0.718341,0.294359,high,0.010546,0.007332,0.013860,...,2.514652,1.290151,0.585978,2.060308,0.000055,0.000035,0.000113,0.000095,0.000039,0.000164
5,0.57,0.200070,0.810237,0.297080,-0.281694,0.798616,low,0.008597,0.006875,0.013943,...,2.111230,1.290151,0.585978,2.060308,0.000039,0.000018,0.000110,0.000095,0.000039,0.000164
6,0.57,0.174062,0.760482,0.297080,-0.281694,0.798616,low,0.009375,0.007606,0.014874,...,1.696193,1.290151,0.585978,2.060308,0.000059,0.000034,0.000117,0.000095,0.000039,0.000164
7,0.45,0.092991,0.757506,0.297080,-0.281694,0.798616,low,0.008714,0.006590,0.014655,...,2.139422,1.290151,0.585978,2.060308,0.000030,0.000012,0.000131,0.000095,0.000039,0.000164


### one factor within subject

In [4]:
np.random.seed(1)

m2 = glam.GLAM()

groups = ['high', 'low']

N = 4

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))

for group in groups:
    
    parameters = dict(v=np.clip(np.random.normal(loc=0.00007, scale=0.00001, size=N), 0.00003, 0.00015),
                      s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                      gamma=np.clip(np.random.normal(loc=gamma[group]['mu'], scale=gamma[group]['sd'], size=N), -1, 1),
                      tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                      t0=np.zeros(N))
    
    m2.simulate_group(kind='individual',
                      parameters=parameters,
                      n_individuals=N,
                      individual_idx=np.arange(N),
                      n_trials=n_trials,
                      n_items=n_items,
                      label=group)
m2.data.rename(columns=dict(condition='group'), inplace=True)

m2.make_model(kind='hierarchical', 
              depends_on=dict(gamma='group'),
              t0_val=0)
m2.fit(method='ADVI', n_advi=10000)

Generating hierarchical model for 4 subjects...
Fitting 1 model(s) using ADVI...
  Fitting model 1 of 1...


Average Loss = 3,759: 100%|██████████| 10000/10000 [00:33<00:00, 295.17it/s]


/!\ Automatically setting parameter precision...


In [5]:
estimates = glam.utils.get_estimates(m2)
estimates

/!\ Automatically setting parameter precision...


,gamma,gamma_hpd_2.5,gamma_hpd_97.5,gamma_mu,gamma_mu_hpd_2.5,gamma_mu_hpd_97.5,group,s,s_hpd_2.5,s_hpd_97.5,...,tau_hpd_97.5,tau_mu,tau_mu_hpd_2.5,tau_mu_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5,v_mu,v_mu_hpd_2.5,v_mu_hpd_97.5
0,-0.37,-0.571643,0.026950,-0.266271,-0.818084,0.310179,high,0.012221,0.007698,0.014201,...,1.887250,1.413067,0.475487,2.558107,0.000069,0.000038,0.000111,0.00011,0.000028,0.000212
1,0.28,-0.047411,0.562137,0.286177,-0.230235,0.703463,low,0.012221,0.007698,0.014201,...,1.887250,1.413067,0.475487,2.558107,0.000069,0.000038,0.000111,0.00011,0.000028,0.000212
2,-0.61,-0.864206,-0.060881,-0.266271,-0.818084,0.310179,high,0.009145,0.007539,0.013890,...,2.271311,1.413067,0.475487,2.558107,0.000048,0.000020,0.000107,0.00011,0.000028,0.000212
3,0.41,-0.035435,0.616196,0.286177,-0.230235,0.703463,low,0.009145,0.007539,0.013890,...,2.271311,1.413067,0.475487,2.558107,0.000048,0.000020,0.000107,0.00011,0.000028,0.000212
4,-0.61,-0.799733,0.125715,-0.266271,-0.818084,0.310179,high,0.011608,0.008833,0.014360,...,2.133780,1.413067,0.475487,2.558107,0.000057,0.000032,0.000099,0.00011,0.000028,0.000212
5,0.51,0.214643,0.679917,0.286177,-0.230235,0.703463,low,0.011608,0.008833,0.014360,...,2.133780,1.413067,0.475487,2.558107,0.000057,0.000032,0.000099,0.00011,0.000028,0.000212
6,-0.74,-0.893974,-0.238872,-0.266271,-0.818084,0.310179,high,0.009534,0.007365,0.013307,...,2.418814,1.413067,0.475487,2.558107,0.000054,0.000019,0.000113,0.00011,0.000028,0.000212
7,0.47,0.184781,0.638434,0.286177,-0.230235,0.703463,low,0.009534,0.007365,0.013307,...,2.418814,1.413067,0.475487,2.558107,0.000054,0.000019,0.000113,0.00011,0.000028,0.000212


## two factors
### two factors between subject

In [6]:
np.random.seed(1)

m3 = glam.GLAM()

N = 5

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))
v = dict(slow=dict(mu=0.00005, sd=0.00001),
         fast=dict(mu=0.00010, sd=0.00001))

for bias in ['high', 'low']:
    for speed in ['slow', 'fast']:
    
        parameters = dict(v=np.clip(np.random.normal(loc=v[speed]['mu'], scale=v[speed]['sd'], size=N), 0.00003, 0.00015),
                          s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                          gamma=np.clip(np.random.normal(loc=gamma[bias]['mu'], scale=gamma[bias]['sd'], size=N), -1, 1),
                          tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                          t0=np.zeros(N))

        m3.simulate_group(kind='individual',
                          parameters=parameters,
                          n_individuals=N,
                          n_trials=n_trials,
                          n_items=n_items,
                          label=bias + '_' + speed)

m3.data['bias'], m3.data['speed'] = m3.data['condition'].str.split('_', 1).str

m3.make_model(kind='hierarchical', 
              depends_on=dict(gamma='bias',
                              v='speed'),
              t0_val=0)
m3.fit(method='ADVI', n_advi=10000)

Generating hierarchical model for 20 subjects...
Fitting 1 model(s) using ADVI...
  Fitting model 1 of 1...


Average Loss = 9,258.1: 100%|██████████| 10000/10000 [01:05<00:00, 152.23it/s]


/!\ Automatically setting parameter precision...


In [7]:
estimates = glam.utils.get_estimates(m3)
estimates

/!\ Automatically setting parameter precision...


,bias,gamma,gamma_hpd_2.5,gamma_hpd_97.5,gamma_mu,gamma_mu_hpd_2.5,gamma_mu_hpd_97.5,s,s_hpd_2.5,s_hpd_97.5,...,tau_hpd_97.5,tau_mu,tau_mu_hpd_2.5,tau_mu_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5,v_mu,v_mu_hpd_2.5,v_mu_hpd_97.5
0,high,-0.27,-0.738585,0.193864,-0.271516,-0.596289,0.116849,0.009860,0.005582,0.014138,...,2.304354,1.302052,0.689822,1.875212,0.000038,0.000016,0.000150,0.000094,0.000037,0.000156
1,high,-0.68,-0.907803,-0.042142,-0.271516,-0.596289,0.116849,0.009488,0.006793,0.015133,...,2.156535,1.302052,0.689822,1.875212,0.000047,0.000011,0.000112,0.000094,0.000037,0.000156
2,high,-0.43,-0.951174,0.475633,-0.271516,-0.596289,0.116849,0.008447,0.006195,0.014258,...,2.673954,1.302052,0.689822,1.875212,0.000017,0.000005,0.000183,0.000094,0.000037,0.000156
3,high,-0.57,-0.923775,0.017506,-0.271516,-0.596289,0.116849,0.010180,0.007368,0.014395,...,2.402738,1.302052,0.689822,1.875212,0.000038,0.000010,0.000126,0.000094,0.000037,0.000156
4,high,-0.59,-0.774269,0.145897,-0.271516,-0.596289,0.116849,0.010196,0.007711,0.014197,...,2.473145,1.302052,0.689822,1.875212,0.000066,0.000019,0.000109,0.000094,0.000037,0.000156
5,high,-0.35,-0.629879,0.048439,-0.271516,-0.596289,0.116849,0.011117,0.007267,0.014486,...,2.099249,1.302052,0.689822,1.875212,0.000069,0.000042,0.000124,0.000099,0.000049,0.000152
6,high,-0.33,-0.573609,0.032737,-0.271516,-0.596289,0.116849,0.008111,0.006820,0.014485,...,1.847508,1.302052,0.689822,1.875212,0.000090,0.000055,0.000134,0.000099,0.000049,0.000152
7,high,-0.17,-0.373668,0.208868,-0.271516,-0.596289,0.116849,0.010263,0.006640,0.014193,...,2.284293,1.302052,0.689822,1.875212,0.000065,0.000039,0.000118,0.000099,0.000049,0.000152
8,high,-0.21,-0.416247,0.108710,-0.271516,-0.596289,0.116849,0.008991,0.006570,0.014557,...,2.163490,1.302052,0.689822,1.875212,0.000091,0.000053,0.000126,0.000099,0.000049,0.000152
9,high,-0.29,-0.527699,-0.047933,-0.271516,-0.596289,0.116849,0.008493,0.003369,0.015354,...,1.705380,1.302052,0.689822,1.875212,0.000110,0.000074,0.000155,0.000099,0.000049,0.000152


### two factors within subject

In [8]:
np.random.seed(1)

m4 = glam.GLAM()

N = 5

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))
v = dict(slow=dict(mu=0.00005, sd=0.00001),
         fast=dict(mu=0.00010, sd=0.00001))

for bias in ['high', 'low']:
    for speed in ['slow', 'fast']:
    
        parameters = dict(v=np.clip(np.random.normal(loc=v[speed]['mu'], scale=v[speed]['sd'], size=N), 0.00003, 0.00015),
                          s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                          gamma=np.clip(np.random.normal(loc=gamma[bias]['mu'], scale=gamma[bias]['sd'], size=N), -1, 1),
                          tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                          t0=np.zeros(N))

        m4.simulate_group(kind='individual',
                          parameters=parameters,
                          individual_idx=np.arange(N),
                          n_individuals=N,
                          n_trials=n_trials,
                          n_items=n_items,
                          label=bias + '_' + speed)

m4.data['bias'], m4.data['speed'] = m4.data['condition'].str.split('_', 1).str

m4.make_model(kind='hierarchical', 
              depends_on=dict(gamma='bias',
                              v='speed'),
              t0_val=0)
m4.fit(method='ADVI', n_advi=10000)

Generating hierarchical model for 5 subjects...
Fitting 1 model(s) using ADVI...
  Fitting model 1 of 1...


Average Loss = 9,186.2: 100%|██████████| 10000/10000 [01:02<00:00, 160.94it/s]


/!\ Automatically setting parameter precision...


In [9]:
estimates = glam.utils.get_estimates(m4)
estimates

/!\ Automatically setting parameter precision...


,bias,gamma,gamma_hpd_2.5,gamma_hpd_97.5,gamma_mu,gamma_mu_hpd_2.5,gamma_mu_hpd_97.5,s,s_hpd_2.5,s_hpd_97.5,...,tau_hpd_97.5,tau_mu,tau_mu_hpd_2.5,tau_mu_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5,v_mu,v_mu_hpd_2.5,v_mu_hpd_97.5
0,high,-0.43,-0.702683,0.054109,-0.22293,-0.696343,0.273205,0.008518,0.006987,0.012791,...,1.858265,1.228171,0.43751,2.122499,0.000060,0.000026,0.000112,0.000106,0.000030,0.000203
1,low,0.68,0.474819,0.840415,0.44751,0.082402,0.802591,0.008518,0.006987,0.012791,...,1.858265,1.228171,0.43751,2.122499,0.000060,0.000026,0.000112,0.000106,0.000030,0.000203
2,high,-0.43,-0.702683,0.054109,-0.22293,-0.696343,0.273205,0.008518,0.006987,0.012791,...,1.858265,1.228171,0.43751,2.122499,0.000068,0.000044,0.000126,0.000104,0.000035,0.000189
3,low,0.68,0.474819,0.840415,0.44751,0.082402,0.802591,0.008518,0.006987,0.012791,...,1.858265,1.228171,0.43751,2.122499,0.000068,0.000044,0.000126,0.000104,0.000035,0.000189
4,high,-0.41,-0.699061,-0.053360,-0.22293,-0.696343,0.273205,0.008399,0.007293,0.013229,...,1.694323,1.228171,0.43751,2.122499,0.000035,0.000014,0.000110,0.000106,0.000030,0.000203
5,low,0.62,0.406750,0.836952,0.44751,0.082402,0.802591,0.008399,0.007293,0.013229,...,1.694323,1.228171,0.43751,2.122499,0.000035,0.000014,0.000110,0.000106,0.000030,0.000203
6,high,-0.41,-0.699061,-0.053360,-0.22293,-0.696343,0.273205,0.008399,0.007293,0.013229,...,1.694323,1.228171,0.43751,2.122499,0.000094,0.000056,0.000128,0.000104,0.000035,0.000189
7,low,0.62,0.406750,0.836952,0.44751,0.082402,0.802591,0.008399,0.007293,0.013229,...,1.694323,1.228171,0.43751,2.122499,0.000094,0.000056,0.000128,0.000104,0.000035,0.000189
8,high,-0.28,-0.505381,0.266665,-0.22293,-0.696343,0.273205,0.009020,0.006849,0.012703,...,1.786749,1.228171,0.43751,2.122499,0.000049,0.000012,0.000117,0.000106,0.000030,0.000203
9,low,0.60,0.383112,0.777828,0.44751,0.082402,0.802591,0.009020,0.006849,0.012703,...,1.786749,1.228171,0.43751,2.122499,0.000049,0.000012,0.000117,0.000106,0.000030,0.000203


# individual models
## one factor
### one factor between subject

In [10]:
np.random.seed(110101)

m5 = glam.GLAM()

groups = ['high', 'low']

N = dict(high=2, low=2)

n_trials = 50
n_items = 4

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))

for group in groups:

    parameters = dict(v=np.clip(np.random.normal(loc=0.00012, scale=0.00001, size=N[group]), 0.00003, 0.00015),
                      s=np.clip(np.random.normal(loc=0.01, scale=0.001, size=N[group]), 0.005, 0.011),
                      gamma=np.clip(np.random.normal(loc=gamma[group]['mu'], scale=gamma[group]['sd'], size=N[group]), -1, 1),
                      tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N[group]), 0.2, 2),
                      t0=np.zeros(N[group]))
    
    m5.simulate_group(kind='individual',
                      parameters=parameters,
                      n_individuals=N[group],
                      n_trials=n_trials,
                      n_items=n_items,
                      label=group)
m5.data.rename(columns=dict(condition='group'), inplace=True)

m5.make_model(kind='individual', 
              depends_on=dict(gamma='group'),
              t0_val=0)
m5.fit(method='NUTS', n_samples=1000, chains=2, njobs=1)

Generating single subject models for 4 subjects...
Fitting 4 model(s) using NUTS...
  Fitting model 1 of 4...


/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/model.py:384: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(var.dtype, float):
100%|██████████| 1500/1500 [00:07<00:00, 198.93it/s]
/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/model.py:384: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(var.dtype, float):


  Fitting model 2 of 4...


100%|██████████| 1500/1500 [00:08<00:00, 174.16it/s]


  Fitting model 3 of 4...


 99%|█████████▉| 1484/1500 [00:12<00:00, 118.75it/s]/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/step_methods/hmc/nuts.py:451: UserWarning: The acceptance probability in chain 0 does not match the target. It is 0.8823091170372115, but should be close to 0.8. Try to increase the number of tuning steps.
  % (self._chain_id, mean_accept, target_accept))
/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/step_methods/hmc/nuts.py:467: UserWarning: Chain 0 contains 5 diverging samples after tuning. If increasing `target_accept` does not help try to reparameterize.
  % (self._chain_id, n_diverging))
100%|██████████| 1500/1500 [00:12<00:00, 119.14it/s]


  Fitting model 4 of 4...


100%|█████████▉| 1494/1500 [00:11<00:00, 125.40it/s]/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/step_methods/hmc/nuts.py:451: UserWarning: The acceptance probability in chain 0 does not match the target. It is 0.8888512541020894, but should be close to 0.8. Try to increase the number of tuning steps.
  % (self._chain_id, mean_accept, target_accept))
100%|██████████| 1500/1500 [00:11<00:00, 125.41it/s]


/!\ Automatically setting parameter precision...


In [11]:
estimates = glam.utils.get_estimates(m5)
estimates

/!\ Automatically setting parameter precision...


,gamma,gamma_hpd_2.5,gamma_hpd_97.5,group,s,s_hpd_2.5,s_hpd_97.5,subject,t0,t0_hpd_2.5,t0_hpd_97.5,tau,tau_hpd_2.5,tau_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5
0,-0.01,-0.078698,0.131467,high,0.008112,0.007376,0.010624,0,0.0,0.0,0.0,0.89,0.532412,1.580414,0.000103,0.000093,0.000121
1,-0.12,-0.327661,0.016686,high,0.011336,0.010173,0.014568,1,0.0,0.0,0.0,0.83,0.459217,1.601896,0.000140,0.000113,0.000157
2,0.71,0.419790,0.949534,low,0.008921,0.007683,0.011766,2,0.0,0.0,0.0,0.70,0.403113,1.400519,0.000134,0.000114,0.000153
3,0.59,0.468020,0.859197,low,0.008759,0.006926,0.010528,3,0.0,0.0,0.0,0.54,0.291806,1.277463,0.000123,0.000103,0.000140


### one factor within subject

In [12]:
np.random.seed(10101)

m6 = glam.GLAM()

groups = ['high', 'low']

N = 2

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))

for group in groups:
    
    parameters = dict(v=np.clip(np.random.normal(loc=0.00007, scale=0.00001, size=N), 0.00003, 0.00015),
                      s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                      gamma=np.clip(np.random.normal(loc=gamma[group]['mu'], scale=gamma[group]['sd'], size=N), -1, 1),
                      tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                      t0=np.zeros(N))
    
    m6.simulate_group(kind='individual',
                      parameters=parameters,
                      n_individuals=N,
                      individual_idx=np.arange(N),
                      n_trials=n_trials,
                      n_items=n_items,
                      label=group)
m6.data.rename(columns=dict(condition='group'), inplace=True)

m6.make_model(kind='individual', 
              depends_on=dict(gamma='group'),
              t0_val=0)
m6.fit(method='NUTS', n_samples=1000, chains=2, njobs=1)

Generating single subject models for 2 subjects...
Fitting 2 model(s) using NUTS...
  Fitting model 1 of 2...


/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/model.py:384: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(var.dtype, float):
100%|█████████▉| 1499/1500 [02:51<00:00,  8.73it/s]/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/step_methods/hmc/nuts.py:451: UserWarning: The acceptance probability in chain 0 does not match the target. It is 0.9569158157163262, but should be close to 0.8. Try to increase the number of tuning steps.
  % (self._chain_id, mean_accept, target_accept))
100%|██████████| 1500/1500 [02:51<00:00,  8.74it/s]


  Fitting model 2 of 2...


100%|██████████| 1500/1500 [00:12<00:00, 123.70it/s]


/!\ Automatically setting parameter precision...


In [13]:
estimates = glam.utils.get_estimates(m6)
estimates

/!\ Automatically setting parameter precision...


,gamma,gamma_hpd_2.5,gamma_hpd_97.5,group,s,s_hpd_2.5,s_hpd_97.5,subject,t0,t0_hpd_2.5,t0_hpd_97.5,tau,tau_hpd_2.5,tau_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5
0,-0.42,-0.518899,-0.260922,high,0.007956,0.007677,0.009775,0,0.0,0.0,0.0,1.08,0.766915,1.815392,0.000069,0.000060,0.000076
1,0.31,0.187597,0.542445,low,0.007956,0.007677,0.009775,0,0.0,0.0,0.0,1.08,0.766915,1.815392,0.000069,0.000060,0.000076
2,-0.34,-0.440601,-0.233073,high,0.007381,0.006896,0.009052,1,0.0,0.0,0.0,0.90,0.568549,1.661806,0.000061,0.000053,0.000068
3,0.57,0.276860,0.807638,low,0.007381,0.006896,0.009052,1,0.0,0.0,0.0,0.90,0.568549,1.661806,0.000061,0.000053,0.000068


## two factors
### two factors between subject

In [14]:
np.random.seed(101110)

m7 = glam.GLAM()

N = 1

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))
v = dict(slow=dict(mu=0.00005, sd=0.00001),
         fast=dict(mu=0.00010, sd=0.00001))

for bias in ['high', 'low']:
    for speed in ['slow', 'fast']:
    
        parameters = dict(v=np.clip(np.random.normal(loc=v[speed]['mu'], scale=v[speed]['sd'], size=N), 0.00003, 0.00015),
                          s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                          gamma=np.clip(np.random.normal(loc=gamma[bias]['mu'], scale=gamma[bias]['sd'], size=N), -1, 1),
                          tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                          t0=np.zeros(N))

        m7.simulate_group(kind='individual',
                          parameters=parameters,
                          n_individuals=N,
                          n_trials=n_trials,
                          n_items=n_items,
                          label=bias + '_' + speed)

m7.data['bias'], m7.data['speed'] = m7.data['condition'].str.split('_', 1).str

m7.make_model(kind='individual', 
              depends_on=dict(gamma='bias',
                              v='speed'),
              t0_val=0)
m7.fit(method='NUTS', n_samples=1000, chains=2, njobs=1)

Generating single subject models for 4 subjects...
Fitting 4 model(s) using NUTS...
  Fitting model 1 of 4...


/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/model.py:384: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(var.dtype, float):
100%|██████████| 1500/1500 [00:08<00:00, 176.72it/s]


  Fitting model 2 of 4...


100%|██████████| 1500/1500 [00:08<00:00, 177.02it/s]


  Fitting model 3 of 4...


100%|██████████| 1500/1500 [00:10<00:00, 137.99it/s]


  Fitting model 4 of 4...


 99%|█████████▉| 1486/1500 [00:09<00:00, 159.52it/s]/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/step_methods/hmc/nuts.py:467: UserWarning: Chain 0 contains 1 diverging samples after tuning. If increasing `target_accept` does not help try to reparameterize.
  % (self._chain_id, n_diverging))
100%|██████████| 1500/1500 [00:09<00:00, 159.66it/s]


/!\ Automatically setting parameter precision...


In [15]:
estimates = glam.utils.get_estimates(m7)
estimates

/!\ Automatically setting parameter precision...


,bias,gamma,gamma_hpd_2.5,gamma_hpd_97.5,s,s_hpd_2.5,s_hpd_97.5,speed,subject,t0,t0_hpd_2.5,t0_hpd_97.5,tau,tau_hpd_2.5,tau_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5
0,high,-0.26,-0.387705,-0.140573,0.006779,0.006045,0.009372,slow,0,0.0,0.0,0.0,0.94,0.490019,2.549092,0.000053,0.000040,0.000059
1,high,-0.36,-0.487312,-0.264656,0.010983,0.008615,0.012985,fast,1,0.0,0.0,0.0,1.12,0.510929,2.081349,0.000103,0.000079,0.000119
2,low,0.53,0.448996,0.908099,0.006522,0.005725,0.008032,slow,2,0.0,0.0,0.0,0.75,0.644988,4.798485,0.000032,0.000027,0.000040
3,low,0.45,0.327710,0.535993,0.008065,0.006734,0.010098,fast,3,0.0,0.0,0.0,1.10,0.571657,1.747990,0.000111,0.000096,0.000126


### two factors within subject

In [16]:
np.random.seed(1011)

m8 = glam.GLAM()

N = 4

n_trials = 50
n_items = 5

gamma = dict(high=dict(mu=-0.3, sd=0.1),
             low=dict(mu=0.5, sd=0.1))
v = dict(slow=dict(mu=0.00005, sd=0.00001),
         fast=dict(mu=0.00010, sd=0.00001))

for bias in ['high', 'low']:
    for speed in ['slow', 'fast']:
    
        parameters = dict(v=np.clip(np.random.normal(loc=v[speed]['mu'], scale=v[speed]['sd'], size=N), 0.00003, 0.00015),
                          s=np.clip(np.random.normal(loc=0.008, scale=0.001, size=N), 0.005, 0.011),
                          gamma=np.clip(np.random.normal(loc=gamma[bias]['mu'], scale=gamma[bias]['sd'], size=N), -1, 1),
                          tau=np.clip(np.random.normal(loc=0.9, scale=0.1, size=N), 0.2, 2),
                          t0=np.zeros(N))

        m8.simulate_group(kind='individual',
                          parameters=parameters,
                          individual_idx=np.arange(N),
                          n_individuals=N,
                          n_trials=n_trials,
                          n_items=n_items,
                          label=bias + '_' + speed)

m8.data['bias'], m8.data['speed'] = m8.data['condition'].str.split('_', 1).str

m8.make_model(kind='individual', 
              depends_on=dict(gamma='bias',
                              v='speed'),
              t0_val=0)
m8.fit(method='NUTS', n_samples=1000, chains=2, njobs=1)

Generating single subject models for 4 subjects...
Fitting 4 model(s) using NUTS...
  Fitting model 1 of 4...


/Users/armin/anaconda3/lib/python3.5/site-packages/pymc3/model.py:384: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(var.dtype, float):
100%|██████████| 1500/1500 [00:15<00:00, 97.07it/s]


  Fitting model 2 of 4...


100%|██████████| 1500/1500 [00:16<00:00, 93.08it/s]


  Fitting model 3 of 4...


100%|██████████| 1500/1500 [00:13<00:00, 107.91it/s]


  Fitting model 4 of 4...


100%|██████████| 1500/1500 [00:14<00:00, 102.20it/s]


/!\ Automatically setting parameter precision...


In [17]:
estimates = glam.utils.get_estimates(m8)
estimates

/!\ Automatically setting parameter precision...


,bias,gamma,gamma_hpd_2.5,gamma_hpd_97.5,s,s_hpd_2.5,s_hpd_97.5,speed,subject,t0,t0_hpd_2.5,t0_hpd_97.5,tau,tau_hpd_2.5,tau_hpd_97.5,v,v_hpd_2.5,v_hpd_97.5
0,high,-0.32,-0.382969,-0.268957,0.007792,0.007637,0.009240,slow,0,0.0,0.0,0.0,1.07,0.793006,1.465375,0.000052,0.000047,0.000057
1,low,0.47,0.323811,0.575740,0.007792,0.007637,0.009240,slow,0,0.0,0.0,0.0,1.07,0.793006,1.465375,0.000052,0.000047,0.000057
2,high,-0.32,-0.382969,-0.268957,0.007792,0.007637,0.009240,fast,0,0.0,0.0,0.0,1.07,0.793006,1.465375,0.000082,0.000074,0.000091
3,low,0.47,0.323811,0.575740,0.007792,0.007637,0.009240,fast,0,0.0,0.0,0.0,1.07,0.793006,1.465375,0.000082,0.000074,0.000091
4,high,-0.29,-0.342759,-0.244905,0.008079,0.007931,0.009606,slow,1,0.0,0.0,0.0,0.88,0.658079,1.195462,0.000059,0.000051,0.000066
5,low,0.47,0.354573,0.635950,0.008079,0.007931,0.009606,slow,1,0.0,0.0,0.0,0.88,0.658079,1.195462,0.000059,0.000051,0.000066
6,high,-0.29,-0.342759,-0.244905,0.008079,0.007931,0.009606,fast,1,0.0,0.0,0.0,0.88,0.658079,1.195462,0.000112,0.000101,0.000119
7,low,0.47,0.354573,0.635950,0.008079,0.007931,0.009606,fast,1,0.0,0.0,0.0,0.88,0.658079,1.195462,0.000112,0.000101,0.000119
8,high,-0.33,-0.390100,-0.249192,0.009159,0.008217,0.009973,slow,2,0.0,0.0,0.0,0.86,0.664092,1.207856,0.000050,0.000042,0.000057
9,low,0.49,0.352713,0.662462,0.009159,0.008217,0.009973,slow,2,0.0,0.0,0.0,0.86,0.664092,1.207856,0.000050,0.000042,0.000057
